In [2]:
import os
from pathlib import Path

import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import matplotlib.pyplot as plt

%load_ext autoreload

In [7]:
event_file = '/gamma/Dropbox (GIN)/UCL_Data/events_files/unet_abnormal_training/fold_0/events.out.tfevents.1692202260.highdimneuro.8598.0'

# load the file
event_acc = EventAccumulator(event_file)
# print scalars
event_acc.Reload()
scalar_tags = event_acc.Tags()['scalars']

# Print the scalar tag names
print("Scalar Tags:")
for tag in scalar_tags:
    print(tag)
    
filtered_tags = ['epoch_train_loss', 'epoch_train_ctr_loss', 'val_mean_dice', 'val_distance', 'ctr_val_volume']

# do the same for all the tags containing 'best'
best_tags = [tag for tag in scalar_tags if 'best' in tag]
"""
epoch_train_loss
epoch_train_ctr_loss
val_mean_dice
ctr_val_loss
ctr_val_volume
val_distance
val_best_mean_dice
val_best_mean_loss
val_best_mean_distance

train = training
ctr = controls
val = validation
"""
translate_dict = {
    'epoch_train_loss': 'Training Loss',
    'epoch_train_ctr_loss': 'Training Controls Loss',
    'val_mean_dice': 'Validation Mean Dice',
    'ctr_val_loss': 'Validation Controls Loss',
    'ctr_val_volume': 'Validation Controls Volume',
    'val_distance': 'Validation Distance',
    'val_best_mean_dice': 'Validation Best Mean Dice',
    'val_best_mean_loss': 'Validation Best Mean Loss',
    'val_best_mean_distance': 'Validation Best Mean Distance'
}

Scalar Tags:
epoch_train_loss
val_mean_dice
val_distance
val_best_mean_dice
val_best_mean_loss
val_best_mean_distance


In [0]:
# do the same for both tags lists but point to the best value in the plot and display its value
# each tag containing "loss", "distance" or "volume" the best value is the lowest
# create a list like the tags lists but with the best value function to apply
best_func_filtered_tags = [min if 'loss' in tag or 'distance' in tag or 'volume' in tag else max for tag in filtered_tags]
best_func_best_tags = [min if 'loss' in tag or 'distance' in tag or 'volume' in tag else max for tag in best_tags]

# plot each scalar in a different subplot
fig, axs = plt.subplots(len(filtered_tags), 1, figsize=(10, len(filtered_tags)*5))
for i, tag in enumerate(filtered_tags):
    x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
    axs[i].plot(x, y)
    axs[i].set_xlabel('Epoch')
    axs[i].set_ylabel(tag)
    # find the best value
    best_value_func = best_func_filtered_tags[i]
    best_value = best_value_func(y)
    best_epoch = x[y.index(best_value)]
    axs[i].plot(best_epoch, best_value, 'ro')
    axs[i].set_title(f'{tag} (Best value: {best_value:.4f})')
plt.tight_layout()
plt.show()

# plot each scalar in a different subplot
fig, axs = plt.subplots(len(best_tags), 1, figsize=(10, len(best_tags)*5))
for i, tag in enumerate(best_tags):
    x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
    axs[i].plot(x, y)
    axs[i].set_xlabel('Epoch')
    axs[i].set_ylabel(tag)
    # find the best value
    best_value_func = best_func_best_tags[i]
    best_value = best_value_func(y)
    best_epoch = x[y.index(best_value)]
    axs[i].plot(best_epoch, best_value, 'ro')
    axs[i].set_title(f'{tag} (Best value: {best_value:.4f})')
plt.tight_layout()
plt.show()

In [ ]:
# plot each scalar in a different subplot and save each plot as a separate image file
output_folder = Path('/data/Dropbox (GIN)/result_ucl_lesseg')  # replace with your output folder path
for i, tag in enumerate(filtered_tags):
    fig, ax = plt.subplots(figsize=(10, 5))
    x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
    ax.plot(x, y)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(tag)
    # find the best value
    best_value_func = best_func_filtered_tags[i]
    best_value = best_value_func(y)
    best_epoch = x[y.index(best_value)]
    ax.plot(best_epoch, best_value, 'ro')
    ax.set_title(f'Best value: {best_value:.4f}')
    plt.tight_layout()
    plt.savefig(output_folder / f'{tag}.png')
    plt.close(fig)  # close the figure

# do the same for the best_tags list
for i, tag in enumerate(best_tags):
    fig, ax = plt.subplots(figsize=(10, 5))
    x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
    ax.plot(x, y)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(tag)
    # find the best value
    best_value_func = best_func_best_tags[i]
    best_value = best_value_func(y)
    best_epoch = x[y.index(best_value)]
    ax.plot(best_epoch, best_value, 'ro')
    ax.set_title(f'Best value: {best_value:.4f}')
    plt.tight_layout()
    plt.savefig(output_folder / f'{tag}.png')
    plt.close(fig)  # close the figure

In [ ]:
from bcblib.tools.visualisation import create_and_save_plots, load_tensorboard_file

# /data/Dropbox (GIN)/UCL_Data/events_files contains several subfolders contains subfolders with event files
# For each subfolder of the root, load the event file and create and save the plots in /data/Dropbox (GIN)/UCL_Data/events_files/plots/subfolder_name

# best_func_filtered_tags = [min if 'loss' in tag or 'distance' in tag or 'volume' in tag else max for tag in filtered_tags]
best_func_filtered_tags = {tag: min if 'loss' in tag or 'distance' in tag or 'volume' in tag else max for tag in filtered_tags}
# best_func_best_tags = [min if 'loss' in tag or 'distance' in tag or 'volume' in tag else max for tag in best_tags]
best_func_best_tags = {tag: min if 'loss' in tag or 'distance' in tag or 'volume' in tag else max for tag in best_tags}
print(best_func_filtered_tags)
root_folder = Path('/data/Dropbox (GIN)/UCL_Data/events_files')
for subfolder in root_folder.iterdir():
    # root_folder / 'plots' should not be processed
    if subfolder.name == 'plots':
        continue
    print(f'Processing {subfolder.name}')
    # for each subfolder in subfolder
    for subsubfolder in subfolder.iterdir():
        # Load the event file
        event_file_list = list(subsubfolder.rglob('events.out.tfevents.*'))
        # only keep the latest event file using the date of modification
        event_file = max(event_file_list, key=os.path.getmtime)    
        event_acc = load_tensorboard_file(str(event_file))
        # Create the output folder subfolder_name/subsubfolder_name
        output_folder = root_folder / 'plots' / subfolder.name / subsubfolder.name
        output_folder.mkdir(parents=True, exist_ok=True)
        # Create and save the plots
        create_and_save_plots(event_acc, best_func_best_tags, output_folder=output_folder, display_plot=False)

In [24]:
%autoreload 2
from bcblib.tools.visualisation import load_all_event_files, plot_scalar_per_fold
best_tags = ['val_best_mean_dice', 'val_best_mean_loss', 'val_best_mean_distance', 'epoch_train_loss', 'epoch_train_ctr_loss', 'val_mean_dice', 'val_distance', 'ctr_val_volume']
root_folder = Path('/gamma/Dropbox (GIN)/UCL_Data/events_files')
# for each subfolder in root_folder call plot_scalar_per_fold to save each scalar plot in subfolder/global_plots/
best_func_best_tags = {tag: min if 'loss' in tag or 'distance' in tag or 'volume' in tag else max for tag in best_tags}

leading_tag = 'val_best_mean_dice'
# print(best_func_filtered_tags)
for subfolder in root_folder.iterdir():
    # root_folder / 'plots' should not be processed
    if subfolder.name == 'plots':
        continue
    print(f'Processing {subfolder.name}')
    # Load the event files
    fold_data = load_all_event_files(subfolder)
    # Create the output folder subfolder_name/global_plots
    output_folder = root_folder / 'plots' / subfolder.name / 'global_plots'
    output_folder.mkdir(parents=True, exist_ok=True)
    # Create and save the plots
    best_leading_values = {}
    for tag in best_tags:
        if tag not in fold_data['fold_1'].Tags()['scalars']:
            print(f'Tag {tag} not found in {subfolder.name}')
            continue
        if tag == leading_tag:
            best_leading_values = plot_scalar_per_fold(fold_data, tag, best_epochs=None, output_path=output_folder, display_plot=False, best_func=best_func_best_tags)
            print(f'Best leading values for {subfolder.name}: {best_leading_values}')
        else:
            plot_scalar_per_fold(fold_data, tag, best_epochs=best_leading_values, output_path=output_folder, display_plot=False, best_func=best_func_best_tags)

Processing abnormal_training_dice_focal


/home/chrisfoulon/neuro_apps/BCBlib/bcblib/tools/visualisation.py:881: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


KeyError: 'fold_1'

In [29]:
import numpy as np
from bcblib.tools.visualisation import load_all_event_files, plot_scalar_per_fold

best_tags = ['val_best_mean_dice', 'val_best_mean_loss', 'val_best_mean_distance', 'epoch_train_loss', 'epoch_train_ctr_loss', 'val_mean_dice', 'val_distance', 'ctr_val_volume']
root_folder = Path('/gamma/Dropbox (GIN)/UCL_Data/events_files')

best_func_best_tags = {tag: min if 'loss' in tag or 'distance' in tag or 'volume' in tag else max for tag in best_tags}

for subfolder in root_folder.iterdir():
    if subfolder.name == 'plots':
        continue
    print(f'Processing {subfolder.name}')
    fold_data = load_all_event_files(subfolder)
    best_epochs = plot_scalar_per_fold(fold_data, 'val_best_mean_dice', best_epochs=None, output_path=None, display_plot=False, best_func=best_func_best_tags)

    for tag in best_tags:
        if tag not in fold_data['fold_1'].Tags()['scalars']:
            print(f'Tag {tag} not found in {subfolder.name}')
            continue

        values_at_best_epochs = []
        for fold_name, best_epoch in best_epochs.items():
            event_acc = fold_data[fold_name]
            x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
            if best_epoch in x:
                value_at_best_epoch = y[x.index(best_epoch)]
                values_at_best_epochs.append(value_at_best_epoch)

        if values_at_best_epochs:
            mean_value = np.mean(values_at_best_epochs)
            print(f'Mean of {tag} across folds at best epoch: {mean_value}')
            # also print the best value of the best epoch for fold_1 values_at_best_epochs[1]
            print(f'Best value of {tag} at best epoch: {values_at_best_epochs[1]}')
        else:
            # print(f'No values found for {tag} at best epoch')
            continue

Processing abnormal_training_dice_focal
Mean of val_best_mean_dice across folds at best epoch: 0.895162308216095
Best value of val_best_mean_dice at best epoch: 0.907972514629364
Mean of val_best_mean_loss across folds at best epoch: 0.12039990127086639
Best value of val_best_mean_loss at best epoch: 0.10987710952758789
Mean of epoch_train_loss across folds at best epoch: 0.0858351856470108
Best value of epoch_train_loss at best epoch: 0.08223088085651398
Tag epoch_train_ctr_loss not found in abnormal_training_dice_focal
Mean of val_mean_dice across folds at best epoch: 0.895162308216095
Best value of val_mean_dice at best epoch: 0.907972514629364
Mean of val_distance across folds at best epoch: 2.6498131275177004
Best value of val_distance at best epoch: 2.289167642593384
Tag ctr_val_volume not found in abnormal_training_dice_focal
Processing unet_abnormal_training
Mean of val_best_mean_dice across folds at best epoch: 0.8408454418182373
Best value of val_best_mean_dice at best epoch:

In [ ]:
"""
Processing abnormal_training_dice_focal
Mean of val_best_mean_dice across folds at best epoch: 0.895162308216095
Best value of val_best_mean_dice at best epoch: 0.907972514629364
Mean of val_best_mean_loss across folds at best epoch: 0.12039990127086639
Best value of val_best_mean_loss at best epoch: 0.10987710952758789
Mean of epoch_train_loss across folds at best epoch: 0.0858351856470108
Best value of epoch_train_loss at best epoch: 0.08223088085651398
Tag epoch_train_ctr_loss not found in abnormal_training_dice_focal
Mean of val_mean_dice across folds at best epoch: 0.895162308216095
Best value of val_mean_dice at best epoch: 0.907972514629364
Mean of val_distance across folds at best epoch: 2.6498131275177004
Best value of val_distance at best epoch: 2.289167642593384
Tag ctr_val_volume not found in abnormal_training_dice_focal
Processing unet_abnormal_training
Mean of val_best_mean_dice across folds at best epoch: 0.8408454418182373
Best value of val_best_mean_dice at best epoch: 0.8490331768989563
Mean of val_best_mean_loss across folds at best epoch: 0.18258351385593413
Best value of val_best_mean_loss at best epoch: 0.1755388379096985
Mean of epoch_train_loss across folds at best epoch: 0.03638098686933518
Best value of epoch_train_loss at best epoch: 0.03302866220474243
Tag epoch_train_ctr_loss not found in unet_abnormal_training
Mean of val_mean_dice across folds at best epoch: 0.8408454418182373
Best value of val_mean_dice at best epoch: 0.8490331768989563
Mean of val_distance across folds at best epoch: 3.9199737548828124
Best value of val_distance at best epoch: 3.7935774326324463
Tag ctr_val_volume not found in unet_abnormal_training
Processing control_training_wf_div10_dicefocal
Mean of val_best_mean_dice across folds at best epoch: 0.8880412817001343
Best value of val_best_mean_dice at best epoch: 0.8990077972412109
Mean of val_best_mean_loss across folds at best epoch: 0.13077860772609712
Best value of val_best_mean_loss at best epoch: 0.1253492683172226
Mean of epoch_train_loss across folds at best epoch: 0.08331928998231888
Best value of epoch_train_loss at best epoch: 0.07825499773025513
Mean of epoch_train_ctr_loss across folds at best epoch: 0.0009658641996793449
Best value of epoch_train_ctr_loss at best epoch: 0.000912784191314131
Mean of val_mean_dice across folds at best epoch: 0.8880412817001343
Best value of val_mean_dice at best epoch: 0.8990077972412109
Mean of val_distance across folds at best epoch: 2.6695798873901366
Best value of val_distance at best epoch: 2.227639675140381
Mean of ctr_val_volume across folds at best epoch: 0.703695747256279
Best value of ctr_val_volume at best epoch: 0.20308123528957367

Trained with controls, the best fold of the SWIN-UNETR reaches an average Dice Score of 0.8990 and an average HD of 2.2276 voxels in the second fold (Fold_1). The average Dice Score across the folds is 0.8880, and the average HD is 2.6696 voxels. Without controls (abnormal_training_dice_focal), the best fold reaches an average Dice Score of 0.9079 and an average HD of 2.2892 voxels in the second fold (Fold_1). The average Dice Score across the folds is 0.8952, and the average HD is 2.6498 voxels. The U-Net without augmentation reaches an average Dice Score of 0.8490 and an average HD of 3.7936 voxels in the second fold (Fold_1). The average Dice Score across the folds is 0.8408, and the average HD is 3.9200 voxels.
"""